In [2]:
from langchain import OpenAI, VectorDBQA, LLMChain
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

In [3]:
import sys
sys.path.append(r"C:\Users\ELAFACRB1\Codice\GitHub\media-chat-service\src\embedding")
from utils import ChromaDBManager, LangChainAI, QDrantDBManager, EmbeddingFunction

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [15]:
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k-0613")

In [16]:
embedding = EmbeddingFunction('openAI').embedder
vectore_store=qdrantClient = QDrantDBManager(
    url="http://ec2-18-209-145-26.compute-1.amazonaws.com:6333/dashboard",
    port=6333,
    collection_name="rio-rag-platform",
    vector_size=1536,
    embedding=embedding,
    record_manager_url=r"sqlite:///C:\Users\ELAFACRB1\Codice\GitHub\media-chat-service\src\embedding\record_manager_cache.sql"
)
vectore_store_client=vectore_store.vector_store

Collection rio-rag-platform already exists!


In [17]:
COLLECTION_NAME="rio-rag-platform"

In [18]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=vectore_store_client.as_retriever(),
    return_source_documents=True,
)

In [19]:
query='puoi consigliarmi qualche caffè letterario? '

In [20]:
qa_chain(query)

{'query': 'puoi consigliarmi qualche caffè letterario? ',
 'result': "Ecco alcuni caffè letterari a Milano che potresti considerare:\n\n1. Lapsus - Via Meda 38, Milano\n2. Long Song Books & Café - Via Stoppani 11, Milano\n3. Mamusca - Via Davanzati 2, Milano\n4. La Scatola Lilla - Après Coup - Via Privata della Braida 5, Milano\n5. Anarres - Via Crespi 11, Milano\n6. Ècate - Via Pomponazzi 9E, Milano\n7. Gogol & Company - Via Savona 101, Milano\n8. Colibrì - Via Laghetto 9/11, Milano\n9. L'Arabesque - Largo Augusto 10, Milano\n10. Librosteria - Via Cesariano 7, Milano\n11. Germi - Via Cicco Simonetta 14/A, Milano\n12. Verso - Corso di Porta Ticinese 40, Milano\n13. Open - Viale Monte Nero 6, Milano\n14. Babitonga Café - Viale Pasubio 5, Milano\n15. RED La Feltrinelli - Piazza Tre Torri, Milano / Piazza Gae Aulenti 1, Milano\n\nSpero che questa lista ti sia utile!",
 'source_documents': [Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMangiare e bere tra i libri: 15 caf

Parent document

In [9]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [10]:
from langchain_community.vectorstores import Qdrant

In [11]:
Qdrant

langchain_community.vectorstores.qdrant.Qdrant

In [12]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [13]:
store = InMemoryStore()

In [14]:
retriever = ParentDocumentRetriever(
    vectorstore=vectore_store.vector_store,
    docstore=store,
    child_splitter=child_splitter,
)

In [15]:
from langchain.document_loaders import RSSFeedLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
urls = ["https://medium.com/feed/tag/quantum-computing"]
loader = RSSFeedLoader(urls=urls)
data = loader.load()

In [16]:
retriever.add_documents(data, ids=None)

ResponseHandlingException: timed out

In [ ]:
retriever

ParentDocumentRetriever(vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x000001AD239F8C90>, docstore=<langchain.storage.in_memory.InMemoryBaseStore object at 0x000001AD247C1E50>, child_splitter=<langchain.text_splitter.RecursiveCharacterTextSplitter object at 0x000001AD251CFF50>)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=vectore_store_client.as_retriever(),
    return_source_documents=True,
)

In [ ]:
qa_chain('whats the most recent computer for quantum computing?')

{'query': 'whats the most recent computer for quantum computing?',
 'result': "The most recent computer for quantum computing is the Sycamore processor developed by Google. It made headlines in 2019 when Google claimed to have achieved quantum supremacy with this processor. However, it's important to note that the field of quantum computing is rapidly evolving, and new advancements are being made by various companies and research institutions.",
 'source_documents': [Document(page_content='The Future of Quantum Computing', metadata={'authors': [], 'description': 'Are you tired of waiting ages for data processing to complete? Frustrated by the limitations of conventional computing power? If so, you’re not alone. In a world where data is king, speed and…', 'doc_id': '7ed35252-4320-4062-a7e2-aada649aa0b1', 'feed': 'https://medium.com/feed/tag/quantum-computing', 'language': 'en', 'link': 'https://medium.com/@krupalu.datta/quantum-supremacy-redefining-speed-and-efficiency-in-data-processin

**Multi-Query Retriever**

If users are asking questions that are complex and require multiple pieces of distinct information to respond

In [ ]:
query='what could be an use of quantum computing in SAFe methodology?'

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
import logging

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectore_store_client.as_retriever(), 
    llm=llm
)

In [ ]:
unique_docs = retriever_from_llm.get_relevant_documents(query=query)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can quantum computing be applied within the context of SAFe methodology?', '2. In what ways could quantum computing enhance the implementation of SAFe methodology?', '3. What potential benefits does quantum computing offer for integrating with SAFe methodology practices?']


In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=retriever_from_llm,
    return_source_documents=True,
)

In [ ]:
qa_chain(query)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can quantum computing be applied within the context of SAFe methodology?', '2. In what ways could quantum computing enhance the implementation of SAFe methodology?', '3. What potential benefits can quantum computing bring to the utilization of SAFe methodology?']


{'query': 'what could be an use of quantum computing in SAFe methodology?',
 'result': 'Quantum computing could potentially be used in SAFe methodology to enhance the development of solutions by improving computational power and efficiency. Quantum algorithms could help in optimizing processes, such as accelerating the development of new pharmaceuticals or solving complex optimization problems in logistics, finance, and manufacturing. This could lead to more efficient and effective implementation of SAFe practices, ultimately improving the quality and speed of solution delivery within the framework.',
 'source_documents': [Document(page_content='The Future of Quantum Computing', metadata={'authors': [], 'description': 'Are you tired of waiting ages for data processing to complete? Frustrated by the limitations of conventional computing power? If so, you’re not alone. In a world where data is king, speed and…', 'doc_id': '7ed35252-4320-4062-a7e2-aada649aa0b1', 'feed': 'https://medium.co

In [ ]:
qa_chain('whats a PI?')

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you explain the concept of PI?', '2. What is the definition of PI?', '3. Could you provide information about PI?']


{'query': 'whats a PI?',
 'result': 'PI stands for Planning Interval. It is a cadence-based event that serves as the heartbeat of the ART (Agile Release Train), aligning all teams on the ART to a shared mission and vision. PI Planning helps in establishing personal communication across all team members and stakeholders, aligning development of business goals with the business context, identifying dependencies, fostering cross-team and cross-ART collaboration, providing architecture and Lean UX guidance, matching demand to capacity, and enabling faster decision-making.',
 'source_documents': [Document(page_content='dependencies 4-24Planning Interval (PI) Planning is a cadence-based event that serves as the  heartbeat of the ART, aligning all teams on the ART to a shared mission  and Vision.What is PI Planning? © Scaled Agile. Inc. Duration Video: The Power of PI Planning 3 https://bit.ly/Video - PowerofPIPlanning min 4-23 Workbook 85 © Scaled Agile, Inc. For personal use only - Rosario 

**Output parser**

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import JsonOutputParser
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

In [ ]:
model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

In [ ]:

class News(BaseModel):
    topic: str = Field(description="The topic of the news")
    titles: List[str] = Field(description="titles of the news")
    descriptions: List[str] = Field(description="full, long text with details news")
    urls: List[str] = Field(description="url of the news if available, empty if not")

    # You can add custom validation logic easily with Pydantic.
    # @validator("setup")
    # def question_ends_with_question_mark(cls, field):
    #     if field[-1] != "?":
    #         raise ValueError("Badly formed question!")
    #     return field

In [ ]:
parser = JsonOutputParser(pydantic_object=News)

In [ ]:
prompt = PromptTemplate(
    template="Answer the user query \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [ ]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"topic": {"title": "Topic", "description": "The topic of the news", "type": "string"}, "titles": {"title": "Titles", "description": "titles of the news", "type": "array", "items": {"type": "string"}}, "descriptions": {"title": "Descriptions", "description": "full, long text with details news", "type": "array", "items": {"type": "string"}}, "urls": {"title": "Urls", "description": "url of the news if available, empty if not", "type": "array", "items": {"type": "string"}}}, "required": ["topic", "titles", "descriptions", 

In [ ]:
prompt

PromptTemplate(input_variables=['query'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"topic": {"title": "Topic", "description": "The topic of the news", "type": "string"}, "titles": {"title": "Titles", "description": "titles of the news", "type": "array", "items": {"type": "string"}}, "descriptions": {"title": "Descriptions", "description": "full, long text with details news", "type": "array", "items": {"type": "string"}}, "urls": {"title": "Urls", "description": "url of the news if available, empty if not", "type": "arr

In [ ]:
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me the last 5 news about quantum computing."})
parser.invoke(output)


In [ ]:
output

{'topic': 'Quantum Computing', 'titles': ['1. ']}

**Structured output**

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import json

In [ ]:
response_schemas = [
    ResponseSchema(
        name="title", 
        description="title of the news"
    ),
    ResponseSchema(
        name="text",
        description="long text (at least 3 row long) of the retrieved news",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions=output_parser.get_format_instructions()

In [ ]:
template = """
    You will be given a series of topic.
    For each topic find a list of the most 5 recent news about that topic.

    {format_instructions}

    Wrap your final output with closed and open brackets (a list of json objects in python format, not markdown).

    INPUT:
    {user_topics}

    YOUR RESPONSE:
"""


In [ ]:
chat_model = ChatOpenAI(temperature=0)

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["user_topics"],
    partial_variables={"format_instructions": format_instructions}
)

_input = prompt.format_prompt(user_topics=['quantum computing', 'cloud computing'])

In [ ]:
output = chat_model(_input.to_messages())
output

AIMessage(content='[\n\t{\n\t\t"title": "Quantum Computing: A New Era of Computing",\n\t\t"text": "Quantum computing is a revolutionary technology that has the potential to solve complex problems that are practically impossible for classical computers. Unlike classical computers that use bits, quantum computers use quantum bits or qubits, which can exist in multiple states simultaneously. This allows quantum computers to perform calculations at a much faster rate than classical computers. Major tech companies like IBM, Google, and Microsoft are investing heavily in quantum computing research, with the goal of achieving quantum supremacy."\n\t},\n\t{\n\t\t"title": "Recent Breakthrough in Quantum Computing Research",\n\t\t"text": "Researchers have made a significant breakthrough in quantum computing research by demonstrating the ability to control qubits at a much higher rate than previously thought possible. This development could pave the way for the creation of more powerful quantum c

In [ ]:
json.loads(output.content)

[{'title': 'Quantum Computing: A New Era of Computing',
  'text': 'Quantum computing is a revolutionary technology that has the potential to solve complex problems that are practically impossible for classical computers. Unlike classical computers that use bits, quantum computers use quantum bits or qubits, which can exist in multiple states simultaneously. This allows quantum computers to perform calculations at a much faster rate than classical computers. Major tech companies like IBM, Google, and Microsoft are investing heavily in quantum computing research, with the goal of achieving quantum supremacy.'},
 {'title': 'Recent Breakthrough in Quantum Computing Research',
  'text': 'Researchers have made a significant breakthrough in quantum computing research by demonstrating the ability to control qubits at a much higher rate than previously thought possible. This development could pave the way for the creation of more powerful quantum computers that can tackle complex problems in fi

With RetrievalQAWithSourcesChain

In [1]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=OpenAI(temperature=0),
    chain_type="stuff",
    retriever=vectore_store_client.as_retriever(),
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["user_topics"],
            partial_variables={"format_instructions": format_instructions},
            return_source_documents=True,
        ),
    },
)

NameError: name 'RetrievalQAWithSourcesChain' is not defined